# Global Variables & Function Definitions

These global definitions require evaluation before running HiDENSEC on any concrete Hi-C map.

## Modules

In [1]:
import numpy as np
import scipy.sparse as sp_sparse
import scipy.signal as sp_signal
import scipy.ndimage as sp_image
from scipy.stats import kendalltau
import scipy.stats as sp_stats

In [2]:
rng = np.random.default_rng()

## Global variables

Load file paths, covariates & centromere locations.

In [766]:
chromosomelocations = np.loadtxt('chromosomelocations.txt')-1
compartmentnames = np.loadtxt('compartmentnames.txt', dtype='str')
compartments = np.loadtxt('compartments.txt', dtype='str')
centromers = np.loadtxt('centromers.txt')-1
newCentromers = np.loadtxt('newCentromers.txt')-1
rPos = np.loadtxt('rPos.txt')-1
fPos = np.loadtxt('fPos.txt')-1
acrox = np.loadtxt('acrox.txt')
corrchromlocations = np.loadtxt('corrchromlocations.txt')-1
centromersCorrected = np.loadtxt('centromersCorrected.txt')-1
covariates = np.loadtxt('covariates.csv', delimiter=',')
excursionLengthsH0 = np.loadtxt('excursionLengthsH0.csv', delimiter=',')
fixCdata = np.loadtxt('fixCdata.csv', delimiter=',', dtype='object')
hiCdata = np.loadtxt('hiCdata.csv', delimiter=',', dtype='object')

## Covariate Correction

Function definitions pertaining to covariate regression.

In [767]:
def partitionByIndex(list, indices):
    return np.split(list, indices)

def partitionByLength(list, lengths):
    return partitionByIndex(list, np.cumsum(lengths))

def findDiploidPeak(data, window=[0.1,1,0.1]):
    clean_data = data[data>0]
    scale = np.median(np.abs(clean_data - np.median(clean_data)))
    modes = []
    for c in np.arange(window[0], window[1] + window[2], window[2]):
        counts, bins = np.histogram(clean_data, bins=np.arange(clean_data.min(), clean_data.max(), c*scale))
        mode_index = int(np.median(np.argmax(counts)))
        modes.append((bins[mode_index] + bins[mode_index+1])/2)
    return np.mean(modes)

def filterPosition(dataset, cutthreshold=75, GCthreshold=0.32, mapthreshold=0.8):
    covariate_part = dataset[:, 1:4].astype(float)
    compartment_part = dataset[:,4]
    return (covariate_part[:,0]>cutthreshold) & (covariate_part[:,1]>GCthreshold) & (covariate_part[:,2]>mapthreshold) & (compartment_part!='A0')

def covariateFilter(dataset, cutthreshold=75, GCthreshold=0.32, mapthreshold=0.8):
    return dataset[filterPosition(dataset, cutthreshold, GCthreshold, mapthreshold)]

def covariateCorrection1(dataset, cutthreshold=75, GCthreshold=0.32, mapthreshold=0.8, midpointwindow=[0.1,1,0.1], neighbourhood=0.3, filterlength=100):
    data = covariateFilter(dataset, cutthreshold, GCthreshold, mapthreshold)
    midpoints = [ findDiploidPeak(data[data[:,4] == compartment_name, 0].astype(float), midpointwindow) for compartment_name in compartmentnames[1:] ]
    inserted_midpoints = np.copy(data[:,-1])
    for j in range(5):
        inserted_midpoints[inserted_midpoints == compartmentnames[1+j]] = midpoints[j]
    compartmentcorrected = 2*data[:,0].astype(float)/inserted_midpoints.astype(float)
    predictions = np.copy(data[:, -1])
    for compartment_name in compartmentnames[1:]:
        compartment_indices = (data[:, -1] == compartment_name) & (np.abs(compartmentcorrected - 2) < neighbourhood)
        compartment_covariates = data[compartment_indices][:, [1,2]].astype(float)
        compartment_values = compartmentcorrected[compartment_indices].astype(float)
        design_mat = np.concatenate( [np.ones([compartment_indices.sum(),1]), compartment_covariates], axis=1 )
        fit = np.linalg.lstsq(design_mat, compartment_values, rcond=None)
        full_compartment_indices = (data[:, -1] == compartment_name)
        full_compartment_covariates = data[full_compartment_indices][:, [1,2]].astype(float)
        full_design_mat = np.concatenate( [np.ones([full_compartment_indices.sum(),1]), full_compartment_covariates], axis=1 )
        predictions[predictions == compartment_name] = full_design_mat @ fit[0]
    fully_corrected_data = 2*compartmentcorrected / predictions.astype(float)
    fully_corrected_data = 2*fully_corrected_data / findDiploidPeak(fully_corrected_data)
    return fully_corrected_data

def attachCovariates(profile):
    return np.concatenate([profile[:,None], np.transpose(covariates), compartments[:,None]], axis=1)

def rawToCorrected(x, filter):
    if type(x) == int:
        return int(np.median(np.argmin(np.abs(x-filter))))
    else:
        return [int(np.median(np.argmin(np.abs(y-filter)))) for y in x]

def xToChi(x):
    return np.argmax(x < corrchromlocations[1:])+1

def dataCcorrector(profile, data_flag, cutthreshold=75, GCthreshold=0.32, mapthreshold=0.8, midpointwindow=[0.1,1,0.1], neighbourhood=0.3, filterlength=100):
    test_data = attachCovariates(profile)
    test_data = covariateFilter(test_data, cutthreshold, GCthreshold, mapthreshold)
    if data_flag == 'HiC':
        reference_data = np.copy(hiCdata)
    elif data_flag == 'FixC':
        reference_data = np.copy(fixCdata)
    else:
        return 'Unknown protocol'
    reference_data = reference_data[:, [4, 1, 2, 3, 0]]
    data = covariateFilter(reference_data, cutthreshold, GCthreshold, mapthreshold)
    midpoints = [ findDiploidPeak(data[data[:,4] == ('"'+compartment_name+'"'), 0].astype(float), midpointwindow) for compartment_name in compartmentnames[1:] ]
    inserted_midpoints = np.copy(data[:,-1])
    for j in range(5):
        inserted_midpoints[inserted_midpoints == ('"'+compartmentnames[1+j]+'"')] = midpoints[j]
    compartmentcorrected = 2*data[:,0].astype(float)/inserted_midpoints.astype(float)
    test_inserted_midpoints = np.copy(test_data[:,-1])
    for j in range(5):
        test_inserted_midpoints[test_inserted_midpoints == (compartmentnames[1+j])] = midpoints[j]
    test_compartmentcorrected = 2*test_data[:,0].astype(float)/test_inserted_midpoints.astype(float)
    predictions = np.copy(test_data[:, -1])
    for compartment_name in compartmentnames[1:]:
        compartment_indices = (data[:, -1] == ('"'+compartment_name+'"')) & (np.abs(compartmentcorrected - 2) < neighbourhood)
        compartment_covariates = data[compartment_indices][:, [1,2]].astype(float)
        compartment_values = compartmentcorrected[compartment_indices].astype(float)
        design_mat = np.concatenate( [np.ones([compartment_indices.sum(),1]), compartment_covariates], axis=1 )
        fit = np.linalg.lstsq(design_mat, compartment_values, rcond=None)
        full_compartment_indices = (test_data[:, -1] == compartment_name)
        full_compartment_covariates = test_data[full_compartment_indices][:, [1,2]].astype(float)
        full_design_mat = np.concatenate( [np.ones([full_compartment_indices.sum(),1]), full_compartment_covariates], axis=1 )
        predictions[predictions == compartment_name] = full_design_mat @ fit[0]
    fully_corrected_data = 2*test_compartmentcorrected / predictions.astype(float)
    fully_corrected_data = 2*medianFilter(fully_corrected_data, 100) / findDiploidPeak(medianFilter(fully_corrected_data, 100))
    return fully_corrected_data

## Copy number & Mixture Proportion Estimation

Function definitions calculating effective copy number profiles.

In [822]:
def medianDeviation(list):
    return np.median(np.abs(list - np.median(list)))

def splitList(list, index):
    return ([list[:index], list[index:]], [index / len(list), 1-index/len(list)])

def findRestrictedMedian(list, candidates):
    epsilon_list = [np.mean(np.abs(list - candidate)) for candidate in candidates]
    minPos = np.argmin(epsilon_list)
    return (candidates[minPos], epsilon_list[minPos], medianDeviation(list))

def findSplit(data, candidates):
    restricted_medians = []
    for j in range(2, len(data) - 1):
        splits = splitList(data, j)
        split_deviations = [np.sum(findRestrictedMedian(split, candidates)[1:]) for split in splits[0]]
        restricted_medians.append(np.array(split_deviations)@np.array(splits[1]))
    return (np.argmin(restricted_medians)+1, np.min(restricted_medians))

def paddedPartition(list, window):
    a = [list[:window] for _ in range(round((window-1)/2))]
    b = [list[j:window+j] for j in range(len(list)-window+1)]
    c = [list[:window] for _ in range(round((window-1)/2))]
    return np.concatenate([a,b,c])

def copyNumberFilter(data, candidates, window=101):
    return [findRestrictedMedian(part, candidates) for part in paddedPartition(data, window)]

def copyNumberVariance(data, candidates, window=101):
    restricted_medians = [findRestrictedMedian(part, candidates) for part in paddedPartition(data, window)]
    return np.mean([z[0]*z[1]/z[2] for z in restricted_medians])

def listCandidates(f, maxPloidy):
    return 2*(1-f)+f*np.arange(1, maxPloidy+1)

def computeChangePoints(profile):
    return np.arange(len(profile)-1)[np.abs(np.diff(profile)) > 0]

def refineChangePoint(data, pt, candidates, window=250, replicates=100):
    w = round((window-1)/2)
    split = findSplit(data[pt-w:pt+w+1], candidates)
    a = pt-w-1+split[0]
    replicates = [findSplit(rng.choice(data[pt-w:pt+w+1], 2*w+1), candidates)[-1] for _ in range(replicates)]
    return (a, (replicates > split[-1]).sum() / len(replicates))

def refineChangePoints(data, pts, candidates, window=250, replicates=100):
    extended_pts = np.concatenate(([0], pts, [len(rPos)-1]))
    parts = [extended_pts[j:j+3] for j in range(len(extended_pts)-2)]
    ws = [ np.min(np.append(np.diff(part)/2, window)) for part in parts ]
    res = []
    for pt, w in zip(pts, ws):
        if w <= 3.5:
            res.append((pt, 0))
        else:
            res.append(refineChangePoint(data, pt, candidates, w, replicates))
    return res

def refineProfile(profile, pts):
    extended_pts = np.concatenate(([0], pts, [len(rPos)]))
    refined_segments = []
    for j in range(len(extended_pts)-1):
        a = int(extended_pts[j])
        b = int(extended_pts[j+1])
        values, counts = np.unique(profile[a:b+1], return_counts=True)
        index = np.argmax(counts)
        refined_segments = np.append(refined_segments, [values[index] for _ in range(b-a)])
    return refined_segments

def flattenExcursions(profile, threshold=200):
    pts = computeChangePoints(profile)
    partitioned_profile = partitionByIndex(profile, pts)
    pos = np.array([ (len(part) <= threshold) for part in partitioned_profile])
    flattened_profile = []
    for j, part in enumerate(partitioned_profile):
        n = len(part)
        if (n<=threshold) & (0 < j < len(partitioned_profile)-1):
            flattened_profile.append(np.concatenate(([partitioned_profile[j-1][0] for _ in range(round(n/2))], [partitioned_profile[j+1][0] for _ in range(round((n+1)/2)) ])))
        else:
            flattened_profile.append(partitioned_profile[j])
    return np.concatenate(flattened_profile)

def rankExcursions(data, refinedProfile, changePoints):
    partitioned_profile = partitionByIndex(refinedProfile, changePoints[:,0].astype(int))
    partitioned_data = partitionByIndex(data, changePoints[:,0].astype(int))
    H0 = data[(1.99 <= refinedProfile) & (refinedProfile <= 2.01)]
    extended_change_points = np.concatenate([[0.5], changePoints[:,1], [0.5]])
    intervalP = [(extended_change_points[j] + extended_change_points[j+1]) for j in range(len(extended_change_points)-1)]
    def extractBulk(list):
        n = np.max([round(0.2*len(list)),1])
        return list[n:-n]
    def findClosestDiploidBlock(blocks, k):
        a, b = np.nonzero(blocks < k)[0], np.nonzero(blocks > k)[0]
        if len(a)>0:
            if len(b)>0:
                return [blocks[a[-1]], blocks[b[0]]]
            else:
                return [blocks[a[-1]]]
        elif len(b)>0:
            return [blocks[b[0]]]
        else:
            return []
    block_ploidy = np.array([part[0] for part in partitioned_profile])
    diploidBlocks = np.nonzero((block_ploidy <= 2.01) & (1.99 <= block_ploidy))[0]
    stat_vals_1 = []
    for part_profile, part_data, j in zip(partitioned_profile, partitioned_data, range(len(partitioned_profile))):
        a = np.mean((excursionLengthsH0 <= len(part_profile)))
        sample_data = np.concatenate([partitioned_data[j] for j in findClosestDiploidBlock(diploidBlocks, j)])
        sampled_data = rng.choice(sample_data, size=(100, len(extractBulk(part_profile))))
        dataH0 = [medianDeviation(sampled_point) for sampled_point in sampled_data]
        threshold = 2 / part_profile[0] * medianDeviation(extractBulk(part_data))
        b = np.mean(dataH0 > threshold)
        stat_vals_1.append(np.max([a,b])**2)
    def uniformSumCDF(x):
        return np.piecewise(x, [x < 0, (0 <= x) & (x < 1), (1 <= x) & (x < 2), 2 <= x], [0, lambda x: x**2/2, lambda x: 1-0.5*(2-x)**2, 1])
    stat_vals_2 = uniformSumCDF(np.array([interval.sum() for interval in intervalP]))
    stat_vals = stat_vals_1 + stat_vals_2
    return uniformSumCDF(stat_vals)

def medianFilter(data, window):
    return np.array([np.median(data[j:j+window]) for j in range(len(data)-window+1)])

def extractExcursions(profile, points):
    extended_points = np.concatenate([[0], points, [len(rPos)]])
    excursion_indices = [[extended_points[j], extended_points[j+1]] for j in range(len(extended_points)-1)]
    excursions = partitionByIndex(profile, points)
    excursion_means = np.array([np.mean(excursion) for excursion in excursions])
    return np.arange(len(excursion_indices))[np.abs(excursion_means - 2) > 0.001]

def excursionLengths(profile, points):
    pos = extractExcursions(profile, points)
    return [len(partitionByIndex(profile, points)[j]) for j in pos]

def benjaminiHochberg(pvalues, alpha):
    ps = np.sort(1-pvalues) - alpha / len(pvalues) * np.arange(1, len(pvalues)+1)
    indices = np.nonzero(ps > 0)[0]
    if len(indices) == 0:
        return 0
    else:
        return 1+indices[0]

def confidenceInterval(data, profile, f, resample_size=100):
    pts = computeChangePoints(profile)
    partitioned_profile = partitionByIndex(profile, pts)
    partitioned_data = partitionByIndex(data, pts)
    pos = extractExcursions(profile, pts)
    partitioned_profile_pos = [partitioned_profile[j] for j in pos]
    excursion_ploidy = np.abs([ 2 - (np.unique(profile_part)[0] - 2*(1-f)) / 2 for profile_part in partitioned_profile_pos ])
    excursion_data = [partitioned_data[j] for j in pos]
    excursion_lengths = np.array([len(profile_part) for profile_part in partitioned_profile_pos])
    resample_sizes = resample_size * excursion_lengths / np.sum(excursion_lengths)
    fluctuations = []
    for data_point, size, ploidy in zip(excursion_data, resample_sizes, excursion_ploidy):
        fluctuations = np.append(fluctuations, [np.abs(np.median(rng.choice(data_point, len(data_point))) - 2) / ploidy for _ in range(int(size))])
    return 2*medianDeviation(fluctuations)

def findThreshold(pvalues, alpha):
    delta = 1
    iter = 1
    sorted_ps = np.sort(pvalues)[::-1]
    while (delta>0) & (iter<np.min([1000, len(pvalues)-1])):
        delta = benjaminiHochberg(sorted_ps[:iter+1], alpha) - benjaminiHochberg(sorted_ps[:iter], alpha)
        iter+=1
    return iter

def estimate_proportion_ploidy(rowsums, maxPloidy):
    data = rowsums
    y = int(corrchromlocations[10])
    smoothed_data = 2*medianFilter(data[:y], 100) / findDiploidPeak(medianFilter(data[:y], 100))
    sigma = []
    for j in tqdm(np.arange(0,1.01,0.01)):
        sigma.append(copyNumberVariance(smoothed_data, 2*(1-j) + j*np.arange(1, maxPloidy+1), 251))
    f = np.arange(0, 1.01, 0.01)[np.argmin(sigma)]
    pi0 = np.array(copyNumberFilter(rowsums, 2*(1-f) + f*np.arange(1, maxPloidy+1), 501))
    xs = np.array(refineChangePoints(data, computeChangePoints(pi0[:,0]), listCandidates(f, maxPloidy), 250))
    pi1 = refineProfile(pi0[:,0], xs[:,0])
    ps = rankExcursions(data, pi1, xs)
    return pi1, f, ps

## Off-diagonal detection

Function definitions detecting fusion events of type (a) and (b)

In [1538]:
def roundToChromosome(x,y):
    chiRange = [xToChi(z) for z in range(x,y+1)]
    chis, counts = np.unique(chiRange, return_counts=True)
    chi = int(np.median(chiRange))
    if np.max(counts / np.sum(counts)) < 0.6:
        acs = []
        for j in chis:
            a = np.max([x, corrchromlocations[j-1]])
            b = np.min([y, corrchromlocations[j] - 1])
            if corrchromlocations[xToChi(b)]-1-b < 50:
                c = corrchromlocations[xToChi(b)] - 1
            else:
                c = b
            acs.append([a,c])
        acs = np.array(acs, dtype=int)
        acs = acs[(acs[:,1] - acs[:,0]) > 200]
        return acs
    else:
        chi_nearest_range = np.arange(corrchromlocations[chi-1], corrchromlocations[chi]-1)
        nearest_x = chi_nearest_range[np.abs(chi_nearest_range - x).argmin()]
        nearest_y = chi_nearest_range[np.abs(chi_nearest_range - y).argmin()]
        if corrchromlocations[xToChi(nearest_x)]-1-nearest_x < 50:
            a = corrchromlocations[xToChi(nearest_x)]-1
        else:
            a = nearest_x
        if corrchromlocations[xToChi(nearest_y)]-1-nearest_x < 50:
            b = corrchromlocations[xToChi(nearest_y)]-1
        else:
            b = nearest_y
        return np.array([a, b], dtype=int)

def scanBlock1(block, ws):
    ws_prod = np.product(ws)
    return sp_signal.oaconvolve(np.ones(ws), block.toarray(), mode='valid') / ws_prod

def scanBlock2(mat, chi1, chi2, ws):
    a = corrchromlocations[chi1-1]
    b = corrchromlocations[chi1]
    c = corrchromlocations[chi2-1]
    d = corrchromlocations[chi2]
    ws_prod = np.product(ws)
    return sp_signal.oaconvolve(np.ones(ws), mat.toarray()[int(a):int(b), int(c):int(d)], mode='valid') / ws_prod

def computeIntensitySquare(mat, x, y, ws):
    a = np.max([x-ws[0]+1, 0])
    b = np.min([x+ws[0]+1, len(rPos)])
    c = np.max([y-ws[1]+1, 0])
    d = np.min([y+ws[1]+1, len(rPos)])
    mat11 = mat[a:x+1, c:y+1]
    mat12 = mat[a:x+1, y+1:d]
    mat21 = mat[x+1:b, c:y+1]
    mat22 = mat[x+1:b, y+1:d]
    return np.array([[mat11.mean(), mat12.mean()], [mat21.mean(), mat22.mean()]])

def detectPattern(intensitySquare):
    normalization_constant = intensitySquare.sum()
    if normalization_constant <= 10**(-8):
        return 0.
    else:
        normalized_densities = np.concatenate(intensitySquare) / normalization_constant
    return np.max(normalized_densities)

def rowPattern(mat, x, ws):
    chi = xToChi(x)
    n = len(rPos)
    w = ws[-1]
    y_indices = np.concatenate([np.arange(0, corrchromlocations[chi-1]), np.arange(corrchromlocations[chi], n)])
    r_block = mat[x-ws[0]+1:x+1, y_indices]
    r = np.concatenate(scanBlock1(r_block, ws))
    s_block = mat[x+1:x+ws[0]+1, y_indices]
    s = np.concatenate(scanBlock1(s_block, ws))
    detected_patterns = []
    for j in range(len(r)-w):
        pattern_mat = np.array([[r[j], r[j+w]], [s[j], s[j+w]]])
        detected_patterns.append(detectPattern(pattern_mat))
    return np.array(detected_patterns)

def testTreeStructure(mat, x, y, w, resamples=100):
    submatrix = mat[np.max([x-w[0],0]):np.min([x+w[0]+1, len(rPos)]), np.max([y-w[1],0]):np.min([y+w[1]+1, len(rPos)])]
    rowMarginals = submatrix.mean(axis=1)
    columnMarginals = submatrix.mean(axis=0)
    rowStatistic = np.mean([np.var(rowMarginals[:w[0]]) * w[0] / (w[0]-1), np.var(rowMarginals[w[0]+1:]) * len(rowMarginals[w[0]+1:]) / (len(rowMarginals[w[0]+1:]) - 1)])
    columnStatistic = np.mean([np.var(columnMarginals[:w[0]]) * w[0] / (w[0]-1), np.var(columnMarginals[w[0]+1:]) * len(columnMarginals[w[0]+1:]) / (len(columnMarginals[w[0]+1:]) - 1)])
    resampledRowStatistic = []
    resampledColumnStatistic = []
    for _ in range(resamples):
        permutedRows = rng.permutation(rowMarginals)
        permutedColumns = rng.permutation(columnMarginals)
        resampledRowStatistic.append(np.mean([np.var(permutedRows[:w[0]]) * w[0] / (w[0]-1), 
                                              np.var(permutedRows[w[0]+1:]) * len(permutedRows[w[0]+1:]) / (len(permutedRows[w[0]+1:]) - 1)]))
        resampledColumnStatistic.append(np.mean([np.var(permutedColumns[:w[0]]) * w[0] / (w[0]-1), 
                                                 np.var(permutedColumns[w[0]+1:]) * len(permutedColumns[w[0]+1:]) / (len(permutedColumns[w[0]+1:]) - 1)]))
    return np.mean([np.mean(resampledRowStatistic > rowStatistic), np.mean(resampledColumnStatistic > columnStatistic)])

def extractButterflyCandidate(mat, chi1, chi2, ws, componentDepth=10):
    blockScan = scanBlock2(mat, chi1, chi2, ws)
    components = sp_image.label(np.where(blockScan > np.median(blockScan), blockScan, 0))[0]
    component_groups, component_counts = np.unique(components, return_counts=True)
    size_threshold = np.sort(component_counts)[-componentDepth]
    large_groups = component_groups[component_counts >= size_threshold]
    large_group_sizes = component_counts[component_counts >= size_threshold]
    group_maxs = [[np.unravel_index(np.where(components == group, blockScan, 0).argmax(), blockScan.shape), round(np.sqrt(size/2))] 
                  for group, size in zip(large_groups, large_group_sizes)]
    return group_maxs

def pickLargeCandidate(mat, x, y, ws):
    localMat = mat[np.max([x-ws[0], 1]):np.min([x+ws[0], len(rPos)]), np.max([y-ws[1], 1]):np.min([y+ws[1], len(rPos)])]
    return np.array(np.unravel_index(localMat.argmax(), localMat.shape)) + np.array([1+x,1+y]) - np.array([1+ws[0], 1+ws[1]])

def butterflySummary(mat, x, y, ws):
    intensitySquare = computeIntensitySquare(mat, x, y, ws)
    if intensitySquare.sum() <= 0:
        intensitySquare = np.zeros([2,2])
    else:
        intensitySquare = intensitySquare / intensitySquare.sum()
    return -np.linalg.det(intensitySquare)

def findButterflySummary(mat, x, y, ws, symmetry=0):
    a = np.max([0, x-ws[0]])
    b = np.min([len(rPos), x+ws[0]+1])
    c = np.max([0, y-ws[1]])
    d = np.min([len(rPos), y+ws[1]+1])
    localMat = np.array([[butterflySummary(mat, j, k, [50, 50]) for k in np.arange(c,d)] for j in np.arange(a,b)])
    if symmetry == 0:
        argmax = np.unravel_index(localMat.argmax(), localMat.shape)
        return [np.array(argmax) + np.array([x, y]) - np.array(ws), localMat[argmax]]
    else:
        argmin = np.unravel_index(localMat.argmin(), localMat.shape)
        return [np.array(argmin) + np.array([x, y]) - np.array(ws), localMat[argmin]]
    return localMat

def treeStatistic(mat, x, y, w):
    a11 = mat[np.max([0, x-w-1]):x, np.max([0, y-w]):y+1][::-1]
    a12 = np.transpose(mat[np.max([0, x-w-1]):x, y+1:np.min([len(rPos), y+w+2])])
    a21 = mat[x:np.min([len(rPos), x+w+1]), np.max([0, y-w]):y+1]
    a22 = np.transpose(mat[x:np.min([len(rPos), x+w+1]), y+1:np.min([len(rPos), y+2+w])][::-1])
    quad_stats = []
    for quad in [a11, a12, a21, a22]:
        w_steps, diags = np.transpose(np.array([[j, np.diagonal(quad.toarray(), j).mean()] for j in np.arange(-w, w+1)]))
        if np.var(diags) <= 0:
            quad_stats.append(0)
        else:
            quad_stats.append(kendalltau(w_steps, diags).statistic)
    tree_stats = (1 + np.array(quad_stats).reshape([2,2]))/2
    return -np.linalg.det(tree_stats)

def detectButterfly(intensitySquare, symmetry=0):
    partition_sum = intensitySquare.sum()
    if partition_sum == 0:
        normalized_square = [[0,0], [0,0]]
    else:
        normalized_square = intensitySquare / partition_sum
    if symmetry == 0:
        return [normalized_square[1,0], normalized_square[0,1]]
    else:
        return [normalized_square[0,0], normalized_square[1,1]]

def testButterflyStructure(mat, x, y, w, resamples=100):
    submatrix = mat[np.max([0, x-w[0]]):np.min([len(rPos), x+w[0]+1]), np.max([0, y-w[1]]):np.min([len(rPos), y+w[1]+1])].toarray()
    def formSubmatrices(matrix):
        return matrix[:w[0], :w[1]], matrix[:w[0], w[1]+1:], matrix[w[0]+1:, :w[1]], matrix[w[0]+1:, w[1]+1:]
    treeStat = np.mean([matrix.var() * matrix.size / (matrix.size - 1) for matrix in formSubmatrices(submatrix)])
    permutedTreeStat = []
    for _ in range(resamples):
        n, m = submatrix.shape
        permutedSubmatrix = submatrix[rng.permutation(n), :][:, rng.permutation(m)]
        permutedTreeStat.append(np.mean([matrix.var() * matrix.size / (matrix.size - 1) for matrix in formSubmatrices(permutedSubmatrix)]))
    return (permutedTreeStat > treeStat).mean()

def testButterflyCandidate(mat, point, ws, diagonalw=10, resamples=10, symmetry=0):
    largeCandidate = pickLargeCandidate(mat, *point, ws)
    chis = np.sort([xToChi(point[0]), xToChi(point[1])])
    if symmetry == 0:
        shift = np.array([1, 0])
        intensity_indices = np.array([[0,0], [0,-1], [1,-1], [1,0], [-1,1], [-1,2], [-2,2], [-2,1]])
    else:
        shift = np.array([0, 0])
        intensity_indices = np.array([[0,0], [0,-1], [-1,-1], [-1,0], [1,1], [1,2], [2,2], [2,1]])
    candidate = findButterflySummary(hic, *largeCandidate, np.round(np.array(ws) / 2).astype(int), symmetry)
    candidate = [candidate[0] + shift, candidate[1]]
    structure = testButterflyStructure(mat, *candidate[0], ws, resamples)
    treeStat = treeStatistic(mat, *candidate[0], diagonalw)
    blockScan = scanBlock2(mat, *chis, [3,3])
    blockScan = blockScan[blockScan > 10**(-10)]
    intensity_ratio_a = np.mean([mat[*index] for index in (candidate[0] + intensity_indices)]) - np.mean(blockScan)
    intensity_ratio_b = np.std(blockScan) * np.sqrt(len(blockScan) / (len(blockScan) - 1))
    intensity_ratio = intensity_ratio_a / intensity_ratio_b
    detected_butterfly = detectButterfly(computeIntensitySquare(mat, *candidate[0], ws), symmetry)
    return [candidate[0][0], candidate[0][1], candidate[1], treeStat, structure, intensity_ratio, detected_butterfly[0], detected_butterfly[1]]

def calibrateButterflyTests(testMat, ratioThreshold=5, symmetry=0):
    flatten_test_matrix = testMat[:,:,[2,3]].reshape(21*21,2)
    flatten_test_matrix = flatten_test_matrix[(np.abs(flatten_test_matrix).mean(axis=1) > 0) & (flatten_test_matrix.mean(axis=1) != np.nan)]
    thetas = np.transpose([flatten_test_matrix.mean(axis=0), flatten_test_matrix.std(axis=0) * np.sqrt(len(flatten_test_matrix) / (len(flatten_test_matrix)-1))])
    calibrated_values = np.zeros((testMat.shape)[:2])
    if symmetry == 0:
        for j in range(len(testMat)):
            for k in range(len(testMat)):
                element = testMat[j, k]
                if (np.array_equal(element, np.zeros(8))) or (element[5] < ratioThreshold):
                    calibrated_values[j,k] = 0
                else:
                    a = 1-sp_stats.norm(loc=thetas[0,0], scale=thetas[0,1]).cdf(element[2])
                    b = 1-sp_stats.norm(loc=thetas[1,0], scale=thetas[1,1]).cdf(element[3])
                    c = element[4]
                    print(element)
                    calibrated_values[j,k] = np.min([a, b, c])
    return calibrated_values

# Analysis

Each of the following sections performs parts of the analysis described in the main paper.

## Load Hi-C Matrix

Replace "hi-c_matrix" with filename of interest

In [761]:
row_indices, col_indices, hic_vals = np.transpose(np.loadtxt('hi-c_matrix'))
relevant_indices = (row_indices < 57509 - 1) & (col_indices < 57509 - 1)
hic = sp_sparse.coo_array((hic_vals[relevant_indices], (row_indices[relevant_indices].astype(int)-1, col_indices[relevant_indices].astype(int)-1)), (57509, 57509))
hic = sp_sparse.csr_array(hic)
del row_indices, col_indices, hic_vals, relevant_indices

## Copy Number & Mixture Proportion Inference

Estimation of effective copy number profile.

In [771]:
# Un-comment the respective lines, if the experimental protocol (Fix-C or Hi-C) of the contact map in question is known.

rowsums = covariateCorrection1(attachCovariates(hic.diagonal()))
# rowsums = dataCcorrector(hic.diagonal(), 'FixC')
# rowsums = dataCcorrector(hic.diagonal(), 'HiC')

In [823]:
pi1, f, ps = estimate_proportion_ploidy(rowsums, 5)

100%|█████████████████████████████████████████| 101/101 [05:30<00:00,  3.27s/it]
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


## Off-diagonal inference

Filter Hi-C matrix by criteria specified in covariate correction

In [1591]:
filtered_hic = hic[rPos,:][:,rPos]

### Type (a) events

In [1593]:
# Extract change point candidates

candidates = np.abs(np.diff(medianFilter(pi1, 50))).nonzero()[0]
candidates = candidates[candidates < corrchromlocations[-1]]
aCandidates = []
extended_candidates = np.concatenate([[0], candidates, [len(rPos)]])
for j in range(1, len(extended_candidates)-1):
    if np.max([extended_candidates[j] - extended_candidates[j-1], extended_candidates[j+1] - extended_candidates[j]]) > 100:
        aCandidates.append(extended_candidates[j])
del candidates, extended_candidates, j

In [1193]:
# Generate row-wise null distributions for extracted change-point candidates

mat = filtered_hic + filtered_hic.transpose() - sp_sparse.diags([filtered_hic.diagonal()], [0])
ws = [300, 300]
testPoints = np.concatenate([aCandidates, acrox]).astype(int)
aRow = []
for testPoint in testPoints:
    if testPoint < np.min(ws):
        aRow.append(rowPattern(mat, testPoint, [testPoint, testPoint]))
    else:
        aRow.append(rowPattern(mat, testPoint, ws))
del mat, ws, testPoints, testPoint

In [1224]:
# Calculate summary statistics for each pair of change-point candidates

mat = filtered_hic + filtered_hic.transpose() - sp_sparse.diags([filtered_hic.diagonal()], [0])
ws = [300, 300]
testPoints = np.concatenate([aCandidates, acrox]).astype(int)
aSummary = np.nan * np.ones([len(testPoints), len(testPoints), 3])
for k in range(len(testPoints)):
    for j in range(len(testPoints)):
        x = testPoints[k]
        y = testPoints[j]
        rowSamples = aRow[k]
        columnSamples = aRow[j]
        target = detectPattern(computeIntensitySquare(mat, x, y, ws))
        if (x>=y) or (xToChi(x) >= xToChi(y)) or (np.max([len(testPoints) - k, len(testPoints) - j]) <= 3):
            aSummary[k, j] = [0, 0, 0]
        else: 
            aSummary[k, j] = [(rowSamples <= target).mean(), (columnSamples <= target).mean(), testTreeStructure(mat, x, y, ws, 100) ]
del mat, ws, testPoints, k ,j

In [1238]:
# Aggregate summary statistics into well-calibrated p-values

testPoints = np.concatenate([aCandidates, acrox]).astype(int)
aP = np.nan * np.ones([len(testPoints), len(testPoints)])
for k in range(len(testPoints)):
    for j in range(len(testPoints)):
        if (k>=j) or (xToChi(testPoints[k]) >= xToChi(testPoints[j])) or (np.abs(testPoints[k] - testPoints[j])<2000):
            aP[k,j] = 0
        else:
            aP[k,j] = np.min(aSummary[k,j])
del testPoints, k, j

### Type (b) events

In [1266]:
# Extract candidate points

bCandidates = np.nan * np.ones([21, 19 + 2, 3])
for chi1 in range(1, 22):
    for chi2 in range(1, 22):
        if chi1 < chi2:
            [(pt1, pt2), size] = extractButterflyCandidate(filtered_hic, chi1, chi2, [50,50], 1)[0]
            bCandidates[chi1-1, chi2-1] = np.array([pt1, pt2, size])
del chi1, chi2, pt1, pt2, size

In [1499]:
# Compute summary statistics

mat = filtered_hic
ws = [50, 50]
bP = np.nan * np.ones([2, 21, 21, 8])
for par in [0,1]:
    for chi1 in range(1, 22):
        for chi2 in range(1, 22):
            if chi1 >= chi2:
                bP[par, chi1-1, chi2-1] = np.zeros(8)
            else:
                bP[par, chi1-1, chi2-1] = testButterflyCandidate(mat, (corrchromlocations[[chi1-1, chi2-1]]+25+bCandidates[chi1-1, chi2-1, 1]).astype(int), ws, 10, 100, par)
del mat, ws, chi1, chi2, par

/var/folders/7b/x_r4cg2j41xg8t6tyrdmnhtw0000gq/T/ipykernel_41107/1393203806.py:162: RuntimeWarning: Degrees of freedom <= 0 for slice
  treeStat = np.mean([matrix.var() * matrix.size / (matrix.size - 1) for matrix in formSubmatrices(submatrix)])
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/var/folders/7b/x_r4cg2j41xg8t6tyrdmnhtw0000gq/T/ipykernel_41107/1393203806.py:167: RuntimeWarning: Degrees of freedom <= 0 for slice
  permutedTreeStat.append(np.mean([matrix.var() * matrix.size / (matrix.size - 1) for matrix in formSubmatrices(permutedSubmatrix)]))
/var/folders/7b/x_r4cg2j41xg8t6tyrdmnhtw0000gq/T/ipykernel_41107/139

In [1544]:
# Convert summaries into well-calibrated p-values

calibratedbP = np.array([calibrateButterflyTests(bP[0], 5, 0), calibrateButterflyTests(bP[1], 5, 1)])

In [1587]:
# Report events corresponding to significant p-values

candidatesa = np.concatenate([aCandidates, acrox]).astype(int)
candidatesb = np.nan * np.ones([2, len(bP[0]), len(bP[0]), n, 5])
for par in [0,1]:
    for j in range(len(bP[0])):
        for k in range(len(bP[0])):
            candidatesb[par, j, k] = np.array([bP[par, j, k, 0], bP[par, j, k, 1], xToChi(bP[par, j, k, 0]), xToChi(bP[par, j, k, 1]), calibratedbP[par, j, k]])
list = np.nan * np.ones([len(candidatesa), len(candidatesa), 5])
for j in range(len(candidatesa)):
    for k in range(len(candidatesa)):
        list[j,k] = np.array([candidatesa[j], candidatesa[k], xToChi(candidatesa[j]), xToChi(candidatesa[k]), aP[j,k]])
list = np.concatenate([np.concatenate(list), np.concatenate(np.concatenate(candidatesb[0])), np.concatenate(np.concatenate(candidatesb[1]))])
list = list[list[:,-1] > 0]
list = list[list[:,-1].argsort()][::-1]
threshold = findThreshold(list[:,-1], 2)